# FIT5196 Task 2 in Assessment 1
#### Student Name: Sarath Gopinathan
#### Student ID: 30434904

Date: 05/09/2020

Version: 1.4

Environment: Python 3.8.5 and Jupyter notebook

In [ ]:
import pandas as pd
import numpy as np
import langid 
import pandas as pd
import xlrd
from nltk.probability import *
from itertools import chain
import nltk
from nltk.stem import PorterStemmer
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.collocations import *
from nltk.probability import *
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

## Reading all the data from sheets and converting it into the required format for processing.


Tweets format : 

[
{
"date":"2020-03-22",
"tweet" : "fiddling while rome burns, the reboot https://t.co/d0idjzgrh2"
},
{
"date":"2020-03-22",
"tweet" : "tweet 2"
},
{
"date":"2020-03-22",
"tweet" : "tweet 3"
},
....
}
]

In [ ]:
%%time
tweets_list = []
excel_data = pd.ExcelFile('./30434904.xlsx')
for i in range(len(excel_data.sheet_names)):
    df = excel_data.parse(i)
    # Cleaning   
    df.dropna(axis=0,inplace=True,how='all')
    df.dropna(axis=1,inplace=True,how='all')
    if(df.columns[0]!= 'text'):
        df.reset_index(inplace=True)
        df.drop(axis=1,columns='index',inplace=True)
        df.columns= df.iloc[0]
        df.drop(index=0,inplace=True)
        
    for j in range(len(df.get('text'))-1):
            
        #removing 0 with space unicode values from tweets. eg: \u2069
        tw = (str(df.get('text')[j+1]).encode('ascii', 'ignore')).decode("utf-8").lower()  
        
        tweet_language = langid.classify(tw)
            
        if(tweet_language[0] == 'en'):
                
            temp = {"date": df.get('created_at')[j+1][0:10], "tweet": tw}
            
            tweets_list.append(temp)

## Converting it into required format.

New format : 

[{
"date":"2020-03-22",
"data":[
{
"tweet" : "fiddling while rome burns, the reboot https://t.co/d0idjzgrh2"
},
{
"tweet" : "tweet 2"
},
{
"tweet" : "tweet 3"
},
....
]
},
{
"date":"2020-03-22",
"data":[
{
"tweet" : "fiddling while rome burns, the reboot https://t.co/d0idjzgrh2"
},
{
"tweet" : "tweet 2"
},
{
"tweet" : "tweet 3"
},
....
]
}
]

In [ ]:
%%time
date_wise_tweets = []

for k in range(len(tweets_list)):
    
    flag = 0
    
    for l in range(len(date_wise_tweets)):
    
        if(date_wise_tweets[l].get('date') == tweets_list[k].get('date')):
            
            current_tweet = tweets_list[k].get('tweet')
            
            insert_dict = {"tweet": current_tweet}
            
            date_wise_tweets[l].get('data').append(insert_dict)
            
            flag = 1            
        
    if(flag == 0):
        data = []
        
        current_tweet = tweets_list[k].get('tweet')

        insert_dict = {"tweet": current_tweet}

        data.append(insert_dict)

        date_insert_dict = {"date": tweets_list[k].get('date'), "data": data}

        date_wise_tweets.append(date_insert_dict) 

## Tokenisation


In [ ]:
tokenizer= RegexpTokenizer(r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?")
date_wise_tokenisation = []

for i in range(len(date_wise_tweets)):
    
    tokens = []
    
    for j in range(len(date_wise_tweets[i].get('data'))):
        
        tokens.extend(tokenizer.tokenize(date_wise_tweets[i].get('data')[j].get('tweet')))
    
    add_dict = {"date" : date_wise_tweets[i].get('date'), "tokens" : tokens}
    
    date_wise_tokenisation.append(add_dict)


## Stop word removal


In [ ]:
# Stopwords removal

date_wise_after_stop_list = []

stopwords_df = pd.read_csv('./stopwords_en.txt',names=['stop_words'])
stopwords_list = set(stopwords_df['stop_words'])

for i in range(len(date_wise_tokenisation)):
    
    new_tokens = []
    
    for j in range (len(date_wise_tokenisation[i].get('tokens'))):
        
        flag = 0
        
        if date_wise_tokenisation[i].get('tokens')[j].lower() not in stopwords_list:
            
            new_tokens.append(date_wise_tokenisation[i].get('tokens')[j].lower())
    
    add_dict = {"date": date_wise_tokenisation[i].get('date'), "tokens" : new_tokens}
    
    date_wise_after_stop_list.append(add_dict)

## Porter Stemmer


In [ ]:
stemmer = PorterStemmer()
final_tokens =[]


for i in range(len(date_wise_after_stop_list)):
    
    stemmed_list = []
    
    for j in range(len(date_wise_after_stop_list[i].get('tokens'))):

#         print(stemmer.stem(pos_list[i].get('tokens')[j][0]))
        
        stemmed_list.append(stemmer.stem(date_wise_after_stop_list[i].get('tokens')[j]))

    add_dict = {"date" : date_wise_after_stop_list[i].get('date'), "tokens" : stemmed_list}

    final_tokens.append(add_dict)

In [ ]:
for i in range(len(final_tokens)):
    
    less_frequent=FreqDist(final_tokens[i].get('tokens'))
    
    lessFreqWords = set(less_frequent.hapaxes())     
    

## Finiding threshhold words and removing them


In [ ]:
after_freq_tokens = []

for i in range(len(final_tokens)):
    
    new_tokens = []
    
    for j in range (len(final_tokens[i].get('tokens'))):
        
        flag = 0
        
        if final_tokens[i].get('tokens')[j].lower() not in lessFreqWords:
            
            new_tokens.append(final_tokens[i].get('tokens')[j].lower())
    
    add_dict = {"date": final_tokens[i].get('date'), "tokens" : new_tokens}
    
    after_freq_tokens.append(add_dict)

# print(date_wise_after_stop_list[1].get('date'))    
print(len(after_freq_tokens[0].get('tokens')))

In [ ]:
all_tokens = []


for i in range(len(after_freq_tokens)):
    
    all_tokens.extend(set(after_freq_tokens[i].get('tokens')))
 
tokens_fd = FreqDist(all_tokens)

most_common_tokens = tokens_fd.most_common()

threshold_list = []

for j in range(len(most_common_tokens)):
    
    if(most_common_tokens[j][1] > 60):
        
        threshold_list.append(most_common_tokens[j][0])
        
threshold_set = set(threshold_list)

In [ ]:
final_datewise_list = []

for i in range(len(after_freq_tokens)):
    
    new_tokens = []
    
    for j in range (len(after_freq_tokens[i].get('tokens'))):
        
        flag = 0
        
        if after_freq_tokens[i].get('tokens')[j] not in threshold_set:
            
            new_tokens.append(after_freq_tokens[i].get('tokens')[j])
    
    add_dict = {"date": after_freq_tokens[i].get('date'), "tokens" : new_tokens}
    
    final_datewise_list.append(add_dict)

# print(date_wise_after_stop_list[1].get('date'))    
# print(len(final_datewise_list[0].get('tokens')))

## Top 100 uni-grams.


In [ ]:
# Top 100 uni-grams.

top_100_uni_list = []

for i in range(len(final_datewise_list)):
    
    freq_dist = FreqDist(final_datewise_list[i].get('tokens'))
    
    dict = {'date': final_datewise_list[i].get('date'), 'tokens' : freq_dist.most_common(100)}
    
    top_100_uni_list.append(dict)

# print(top_100_uni_list)

## Top 100 bi-gram


In [ ]:
top_100_bi_list = []

for i in range(len(date_wise_tokenisation)):
    
    freq_dist = FreqDist(ngrams(date_wise_tokenisation[i].get('tokens'), n = 2))
    
    dict = {'date': date_wise_tokenisation[i].get('date'), 'tokens' : freq_dist.most_common(100)}
    
    top_100_bi_list.append(dict)

# print(top_100_bi_list)

## Writing files

In [ ]:
with open("./30434904_100uni.txt", "w", encoding="utf-8") as file:
    
    for i in range(len(top_100_uni_list)):
        
        file.writelines(top_100_uni_list[i].get('date') + ":" + str(top_100_uni_list[i].get('tokens')) + "\n")
    

In [ ]:
with open("./30434904_100bi.txt", "w", encoding="utf-8") as file:
    
    for i in range(len(top_100_bi_list)):
        
        file.writelines(top_100_bi_list[i].get('date') + ":" + str(top_100_bi_list[i].get('tokens')) + "\n")

In [ ]:
all_tokens = []

for i in range(len(final_datewise_list)):
    
    all_tokens.extend(set(final_datewise_list[i].get('tokens')))

tokens_fd = FreqDist(all_tokens)

most_common_tokens = tokens_fd.most_common()

threshold_list = []

for j in range(len(most_common_tokens)):
    
    if(most_common_tokens[j][1] < 5):
        
        threshold_list.append(most_common_tokens[j][0])
        
threshold_set = set(threshold_list)

In [ ]:
after_threshold_list = []

for i in range(len(final_datewise_list)):
    
    new_tokens = []
    
    for j in range (len(final_datewise_list[i].get('tokens'))):
        
        flag = 0
        
        if final_datewise_list[i].get('tokens')[j] not in threshold_set:
            
            new_tokens.append(final_datewise_list[i].get('tokens')[j])
    
    add_dict = {"date": final_datewise_list[i].get('date'), "tokens" : new_tokens}
    
    after_threshold_list.append(add_dict)

# print(date_wise_after_stop_list[1].get('date'))    
print(len(after_threshold_list[0].get('tokens')))

## Final vocab list

In [ ]:
final_vocab_list=[]

for i in range(len(after_threshold_list)):
    final_vocab_list.extend(after_threshold_list[i].get('tokens'))
    
    
    

In [ ]:
final_vocab_list=sorted(set(final_vocab_list))
len(final_vocab_list)

In [ ]:
list_words = []

for i in range(len(after_threshold_list)):
    list_words.extend(after_threshold_list[i].get('tokens'))

bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(list_words)
bigram_finder.apply_freq_filter(20)
bigram_finder.apply_word_filter(lambda w: len(w)<3)# length less than 3 is i
top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi,200) # Top-200 bigrams
top_200_bigrams=sorted(top_200_bigrams)

print(top_200_bigrams)

In [ ]:
final_vocab = set()

for i in range(len(top_200_bigrams)):
    bigram_words = top_200_bigrams[i][0] + "_" + top_200_bigrams[i][1]
    final_vocab_list.append(bigram_words)

final_vocab = sorted(set(final_vocab_list))
print(final_vocab)

# Writing vocab

In [ ]:
with open("./30434904_vocab.txt", "w", encoding="utf-8") as file:
    
    for i in range(len(final_vocab)):
        
        file.writelines(final_vocab[i]+ ":" + str(i) + "\n")

In [ ]:
count_vec_list = []
for i in range(len(after_threshold_list)):
     
    freqdist = FreqDist(after_threshold_list[i].get('tokens'))
    
    write_dict = {"date": after_threshold_list[i].get('date'), "tokens" : freqdist.most_common()}
    count_vec_list.append(write_dict)

print(count_vec_list)

## Final vocab list

In [ ]:
with open("./30434904_countVec.txt", "w", encoding="utf-8") as file:

    for i in range(len(count_vec_list)):
        
        file.writelines(count_vec_list[i].get('date') + ",")

        for j in range(len(count_vec_list[i].get('tokens'))):

            for k in range(len(final_vocab)):

                if(count_vec_list[i].get('tokens')[j][0] == final_vocab[k]):

                    file.writelines(str(k) +":" + str(count_vec_list[i].get('tokens')[j][1]) + ",")
        
        file.writelines("\n")
    